In [1]:
import pandas as pd

# Load the dataset (replace 'your_dataset.csv' with the file path)
df = pd.read_csv("./data/raw/dataset.csv")

# Display the first few rows to understand the structure
print(df.head())


  Agency    Agency Type Distribution Channel              Product Name Claim  \
0    JZI       Airlines               Online                Basic Plan    No   
1    EPX  Travel Agency               Online         Cancellation Plan    No   
2    EPX  Travel Agency               Online  2 way Comprehensive Plan    No   
3    EPX  Travel Agency               Online  2 way Comprehensive Plan    No   
4    EPX  Travel Agency               Online         Cancellation Plan    No   

   Duration                Destination  Net Sales  Commission (in value)  \
0     104.0  TAIWAN, PROVINCE OF CHINA       35.0                  12.25   
1      47.0                      CHILE       80.0                   0.00   
2      63.0                      CHINA       35.0                   0.00   
3      39.0                  INDONESIA       20.0                   0.00   
4      21.0                      INDIA       29.0                   0.00   

  Gender   Age  
0      M  34.0  
1    NaN  36.0  
2    NaN  3

In [2]:
# Check for missing values and calculate the percentage
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

# Combine into a single DataFrame for better readability
missing_summary = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percentage
}).sort_values(by='Percentage', ascending=False)

# Display the summary
print(missing_summary)



                       Missing Values  Percentage
Gender                          16962   79.633803
Claim                               1    0.004695
Destination                         1    0.004695
Commission (in value)               1    0.004695
Net Sales                           1    0.004695
Age                                 1    0.004695
Duration                            1    0.004695
Distribution Channel                0    0.000000
Agency Type                         0    0.000000
Agency                              0    0.000000
Product Name                        0    0.000000


In [3]:
# Drop a single column by name
df.dropna()


,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commission (in value),Gender,Age
0,JZI,Airlines,Online,Basic Plan,No,104.0,"TAIWAN, PROVINCE OF CHINA",35.00,12.25,M,34.0
5,JZI,Airlines,Online,Basic Plan,No,2.0,MALAYSIA,18.00,6.30,M,44.0
8,JZI,Airlines,Online,Basic Plan,No,3.0,MALAYSIA,18.00,6.30,M,45.0
9,JWT,Airlines,Online,Value Plan,No,1.0,INDIA,62.00,24.80,M,118.0
15,JZI,Airlines,Online,Basic Plan,No,14.0,HONG KONG,26.00,9.10,F,43.0
...,...,...,...,...,...,...,...,...,...,...,...
21283,C2B,Airlines,Online,Bronze Plan,No,20.0,SINGAPORE,27.00,6.75,F,51.0
21284,C2B,Airlines,Online,Bronze Plan,No,16.0,SINGAPORE,27.00,6.75,F,34.0
21287,JZI,Airlines,Online,Basic Plan,No,15.0,MALAYSIA,18.00,6.30,M,35.0
21293,C2B,Airlines,Online,Silver Plan,No,6.0,SINGAPORE,24.05,6.01,F,27.0


In [4]:
df.dtypes

Agency                    object
Agency Type               object
Distribution Channel      object
Product Name              object
Claim                     object
Duration                 float64
Destination               object
Net Sales                float64
Commission (in value)    float64
Gender                    object
Age                      float64
dtype: object

In [5]:
# Change column data types
df = df.astype({
    'Agency': 'string',
    'Agency Type': 'string',
    'Distribution Channel': 'string',
    'Product Name': 'string',
    'Claim': 'string',
    'Duration': 'float64',
    'Destination': 'string',
    'Net Sales': 'float64',
    'Commission (in value)': 'float64'
})

# Verify the changes
print(df.dtypes)

Agency                   string[python]
Agency Type              string[python]
Distribution Channel     string[python]
Product Name             string[python]
Claim                    string[python]
Duration                        float64
Destination              string[python]
Net Sales                       float64
Commission (in value)           float64
Gender                           object
Age                             float64
dtype: object


In [6]:
df.drop('Agency', axis=1, inplace=True)

In [7]:
print(df.describe())


           Duration     Net Sales  Commission (in value)           Age
count  21299.000000  21299.000000           21299.000000  21299.000000
mean      47.177238     37.586096               7.981518     39.895065
std      112.548079     43.274026              16.760875     13.668370
min       -2.000000   -389.000000               0.000000      1.000000
25%       10.000000     18.000000               0.000000     36.000000
50%       23.000000     26.000000               0.000000     36.000000
75%       53.000000     44.000000               9.100000     42.000000
max     4881.000000    810.000000             283.500000    118.000000


In [9]:
from sklearn.preprocessing import StandardScaler

# Numerical features to scale
numerical_features = ['Duration', 'Net Sales', 'Commission (in value)', 'Age']

# Categorical features to include
categorical_features = [ 'Agency Type', 'Distribution Channel', 'Product Name', 'Destination', 'Claim']

# Scale numerical features
scaler = StandardScaler()
scaled_numerical = scaler.fit_transform(df[numerical_features])

# Create a DataFrame for scaled numerical features
df_scaled_numerical = pd.DataFrame(scaled_numerical, columns=numerical_features)

# Combine scaled numerical and categorical features
df_final = pd.concat([df_scaled_numerical, df[categorical_features].reset_index(drop=True)], axis=1)

# Verify the resulting DataFrame
print(df_final.head())




   Duration  Net Sales  Commission (in value)       Age    Agency Type  \
0  0.504887  -0.059762               0.254675 -0.431303       Airlines   
1 -0.001575   0.980147              -0.476211 -0.284976  Travel Agency   
2  0.140590  -0.059762              -0.476211 -0.284976  Travel Agency   
3 -0.072657  -0.406399              -0.476211 -0.284976  Travel Agency   
4 -0.232593  -0.198417              -0.476211 -0.284976  Travel Agency   

  Distribution Channel              Product Name                Destination  \
0               Online                Basic Plan  TAIWAN, PROVINCE OF CHINA   
1               Online         Cancellation Plan                      CHILE   
2               Online  2 way Comprehensive Plan                      CHINA   
3               Online  2 way Comprehensive Plan                  INDONESIA   
4               Online         Cancellation Plan                      INDIA   

  Claim  
0    No  
1    No  
2    No  
3    No  
4    No  


In [10]:
df_final.dtypes

Duration                        float64
Net Sales                       float64
Commission (in value)           float64
Age                             float64
Agency Type              string[python]
Distribution Channel     string[python]
Product Name             string[python]
Destination              string[python]
Claim                    string[python]
dtype: object